In [5]:
import pandas as pd 
import glob 
import os
PATH = os.getcwd()  
while not os.path.basename(PATH).startswith('Pin_FS_SVM'):
    parent = os.path.dirname(PATH)
    if parent == PATH:  
        break
    PATH = parent
results_dir = os.path.join(PATH, 'src', 'experiment', 'results')
datasets = ['cleveland', 'diabetes', 'ionosphere', 'sonar', 'wdbc', 'colon']
list = [os.path.join(results_dir, f'experiment_results_{dataset}.xlsx') for dataset in datasets]


In [6]:
df_list = [pd.read_excel(file) for file in list]

In [7]:
combined_df = pd.concat(df_list, ignore_index=True)


In [8]:
columns_to_select = combined_df.columns[:8].tolist() + ['Overlap Ratio']
if 'Average train_time' in columns_to_select:
    columns_to_select.remove('Average train_time')
combined_df = combined_df[columns_to_select]
combined_df = combined_df.rename(columns={'Overlap Ratio': ' Average Overlap Ratio'})

In [9]:
combined_df

,Model,Type of dataset,Average Accuracy,Average AUC,Average F1 Score,Average G-Mean,Average num_features,Average Overlap Ratio
0,L1SVM,Not noise,0.838280,0.838476,0.837574,0.833741,8.5,0.7500
1,L1SVM,Noise,0.785591,0.784821,0.784129,0.777550,10.8,0.5556
2,L1SVM,Outlier,0.785484,0.791390,0.786142,0.785037,9.6,0.6000
3,L1SVM,Noise + Outlier,0.734946,0.738004,0.737131,0.734668,11.1,0.6000
4,L2SVM,Not noise,0.838172,0.836778,0.837311,0.831671,13.0,0.5385
...,...,...,...,...,...,...,...,...
163,RFESVM,Noise + Outlier,0.615385,0.655238,0.603067,0.614464,17.0,0.1176
164,MILP1,Not noise,0.757692,0.757500,0.757980,0.746128,8.0,0.3750
165,MILP1,Noise,0.761538,0.725159,0.746158,0.686431,1.0,1.0000
166,MILP1,Outlier,0.738462,0.739643,0.736606,0.727577,13.6,0.1667


In [10]:
# Calculate mean and std, then format as 'mean ± std'
def mean_std_format(x):
    if x.name in ['Average num_features']:
        mean_val = x.mean()
        std_val = x.std()
        return f"{mean_val:.2f} ± {std_val:.2f}"
    else:
        mean_val = x.mean() * 100
        std_val = x.std() * 100
        return f"{mean_val:.2f} ± {std_val:.2f}"

def mean_format(x):
    if x.name in ['Average num_features']:
        mean_val = x.mean()
        return f"{mean_val:.2f}"
    else:
        mean_val = x.mean() * 100
        return f"{mean_val:.2f}"
avg_df_mean = combined_df.groupby(['Model', 'Type of dataset']).agg(
    lambda x: mean_format(x) if x.dtype in ['float64', 'int64'] else x.iloc[0]
).reset_index()

avg_df_mean_std = combined_df.groupby(['Model', 'Type of dataset']).agg(
    lambda x: mean_std_format(x) if x.dtype in ['float64', 'int64'] else x.iloc[0]
).reset_index()


In [11]:
avg_df_mean.to_excel("average_metrics_mean_format_per_model_dataset.xlsx", index=False)


In [12]:
avg_df_mean_std.to_excel("average_metrics_mean_std_format_per_model_dataset.xlsx", index=False)